In [57]:
'''
Loading data from previous module dank_ocr.csv (dank data plus ocr text_from_image)
Tokenize,lemmatize and clean(optional: remove broken links records) data.
'''
import pandas as pd

dank_data = pd.read_csv("./dank_ocr.csv", delimiter = ",")


In [58]:
dank_data.shape

(53948, 17)

In [60]:
dank_data.drop(columns=['words'], axis=1, inplace=True)
dank_data

,author,awards,created_utc,text_from_image,downs,id,is_nsfw,media,subreddit,subscribers,thumbnail.height,thumbnail.thumbnail,thumbnail.width,title,ups,url
0,NaN,[],1.584296e+09,Fail,0,fj5m13,False,https://i.redd.it/t9x5necdnvm41.jpg,r/dank_meme,743116,114.0,default,140.0,USA right now,1,/r/dank_meme/comments/fj5m13/usa_right_now/
1,L71U,[],1.584296e+09,NICE DICK\r\n,0,fj5ixk,False,https://i.redd.it/w6acm47gmvm41.jpg,r/dank_meme,743116,136.0,https://b.thumbs.redditmedia.com/PixKjoPWVw738...,140.0,Ricardo😎,35,/r/dank_meme/comments/fj5ixk/ricardo/
2,Awesome-ness5,[],1.584295e+09,Nvidia's calling on gaming PC owners to\r\nput...,0,fj5bzj,False,https://i.redd.it/bqawbb5gkvm41.jpg,r/dank_meme,743116,140.0,https://b.thumbs.redditmedia.com/Fj8eeEt78J-I6...,140.0,Gamers Assemble,11,/r/dank_meme/comments/fj5bzj/gamers_assemble/
3,NaN,[],1.584295e+09,Fail,0,fj5aab,False,https://youtu.be/DxeVd-3G95M?t=10s,r/dank_meme,743116,105.0,default,140.0,the worst surgery addicted teen yet,0,/r/dank_meme/comments/fj5aab/the_worst_surgery...
4,NaN,[],1.584294e+09,Fail,0,fj57rc,False,https://i.redd.it/mgzxiyo6jvm41.png,r/dank_meme,743116,140.0,default,140.0,Ok boomer,1,/r/dank_meme/comments/fj57rc/ok_boomer/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53943,Abk0306,[],1.584037e+09,Anti vax moms-\r\nYou can't defeat me.\r\nSoci...,0,fhku7w,False,https://i.redd.it/2uuuxic5aam41.jpg,r/memes,9507605,140.0,https://b.thumbs.redditmedia.com/8dS66OUZe0asw...,140.0,Die mother fuckers,42,/r/memes/comments/fhku7w/die_mother_fuckers/
53944,ccanlas808,[],1.584037e+09,NaN,0,fhku7i,False,https://i.redd.it/3h9unvm5aam41.jpg,r/memes,9507605,140.0,default,140.0,“Just checking for COVID-19”,1,/r/memes/comments/fhku7i/just_checking_for_cov...
53945,NaN,[],1.584037e+09,Fail,0,fhku4u,False,https://i.redd.it/pdb35lh4aam41.jpg,r/memes,9507605,114.0,default,140.0,No.,1,/r/memes/comments/fhku4u/no/
53946,M1CKYMC,[],1.584037e+09,ARSQUARE\r\nA 38680707 c\r\nmeme.org\r\n,0,fhku3d,False,https://i.redd.it/amea4y14aam41.jpg,r/memes,9507605,91.0,https://a.thumbs.redditmedia.com/nYZXMaUA7KPZf...,140.0,"No need to panic, yer TP costs the same as thi...",14,/r/memes/comments/fhku3d/no_need_to_panic_yer_...


In [61]:
dank_data.count()

author                 38931
awards                 53948
created_utc            53948
text_from_image        51284
downs                  53948
id                     53948
is_nsfw                53948
media                  53948
subreddit              53948
subscribers            53948
thumbnail.height       53768
thumbnail.thumbnail    53948
thumbnail.width        53768
title                  53947
ups                    53948
url                    53948
dtype: int64

In [63]:
dank_data.insert(4, "words", "Empty") 

In [24]:
for ind in dank_data.index:    
    if dank_data['words'][ind] == 'Empty':
        title = dank_data['title'][ind]
        text_from_image = dank_data['text_from_image'][ind]
        try:
            if text_from_image == 'Fail':
                text_from_image = ''  
            words = title +  text_from_image
        except:
            words = 'Fail'
        dank_data['words'][ind] = words

In [69]:
dank_data.count()

author                 38931
awards                 53948
created_utc            53948
text_from_image        51284
words                  53948
downs                  53948
id                     53948
is_nsfw                53948
media                  53948
subreddit              53948
subscribers            53948
thumbnail.height       53768
thumbnail.thumbnail    53948
thumbnail.width        53768
title                  53947
ups                    53948
url                    53948
dtype: int64

In [71]:
# Source https://towardsdatascience.com/nlp-extracting-the-main-topics-from-your-dataset-using-lda-in-minutes-21486f5aa925
'''
Loading Gensim and nltk libraries
'''

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [25]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/eli/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [73]:
#https://towardsdatascience.com/nlp-extracting-the-main-topics-from-your-dataset-using-lda-in-minutes-21486f5aa925
'''
Write a function to perform the pre processing steps on the entire dataset
'''
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [74]:
dank_data.insert(5, "processed_words", "Empty") 

In [26]:
import pandas as pd
stemmer = SnowballStemmer("english")
  
for ind in dank_data.index:     
    if dank_data['processed_words'][ind] == 'Empty':
        words = dank_data['words'][ind]
        try:
            processed_words = preprocess(words)
        except:
            processed_words = 'Fail'
        dank_data['processed_words'][ind] = processed_words    

In [76]:
dank_data

,author,awards,created_utc,text_from_image,words,processed_words,downs,id,is_nsfw,media,subreddit,subscribers,thumbnail.height,thumbnail.thumbnail,thumbnail.width,title,ups,url
0,NaN,[],1.584296e+09,Fail,USA right now,[right],0,fj5m13,False,https://i.redd.it/t9x5necdnvm41.jpg,r/dank_meme,743116,114.0,default,140.0,USA right now,1,/r/dank_meme/comments/fj5m13/usa_right_now/
1,L71U,[],1.584296e+09,NICE DICK\r\n,Ricardo😎NICE DICK\r\n,"[ricardo, nice, dick]",0,fj5ixk,False,https://i.redd.it/w6acm47gmvm41.jpg,r/dank_meme,743116,136.0,https://b.thumbs.redditmedia.com/PixKjoPWVw738...,140.0,Ricardo😎,35,/r/dank_meme/comments/fj5ixk/ricardo/
2,Awesome-ness5,[],1.584295e+09,Nvidia's calling on gaming PC owners to\r\nput...,Gamers AssembleNvidia's calling on gaming PC o...,"[gamer, assemblenvidia, call, game, owner, sys...",0,fj5bzj,False,https://i.redd.it/bqawbb5gkvm41.jpg,r/dank_meme,743116,140.0,https://b.thumbs.redditmedia.com/Fj8eeEt78J-I6...,140.0,Gamers Assemble,11,/r/dank_meme/comments/fj5bzj/gamers_assemble/
3,NaN,[],1.584295e+09,Fail,the worst surgery addicted teen yet,"[worst, surgeri, addict, teen]",0,fj5aab,False,https://youtu.be/DxeVd-3G95M?t=10s,r/dank_meme,743116,105.0,default,140.0,the worst surgery addicted teen yet,0,/r/dank_meme/comments/fj5aab/the_worst_surgery...
4,NaN,[],1.584294e+09,Fail,Ok boomer,[boomer],0,fj57rc,False,https://i.redd.it/mgzxiyo6jvm41.png,r/dank_meme,743116,140.0,default,140.0,Ok boomer,1,/r/dank_meme/comments/fj57rc/ok_boomer/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53943,Abk0306,[],1.584037e+09,Anti vax moms-\r\nYou can't defeat me.\r\nSoci...,Die mother fuckersAnti vax moms-\r\nYou can't ...,"[mother, fuckersanti, mom, defeat, societi, kn...",0,fhku7w,False,https://i.redd.it/2uuuxic5aam41.jpg,r/memes,9507605,140.0,https://b.thumbs.redditmedia.com/8dS66OUZe0asw...,140.0,Die mother fuckers,42,/r/memes/comments/fhku7w/die_mother_fuckers/
53944,ccanlas808,[],1.584037e+09,NaN,Fail,[fail],0,fhku7i,False,https://i.redd.it/3h9unvm5aam41.jpg,r/memes,9507605,140.0,default,140.0,“Just checking for COVID-19”,1,/r/memes/comments/fhku7i/just_checking_for_cov...
53945,NaN,[],1.584037e+09,Fail,No.,[],0,fhku4u,False,https://i.redd.it/pdb35lh4aam41.jpg,r/memes,9507605,114.0,default,140.0,No.,1,/r/memes/comments/fhku4u/no/
53946,M1CKYMC,[],1.584037e+09,ARSQUARE\r\nA 38680707 c\r\nmeme.org\r\n,"No need to panic, yer TP costs the same as thi...","[need, panic, cost, piecearsquar, meme]",0,fhku3d,False,https://i.redd.it/amea4y14aam41.jpg,r/memes,9507605,91.0,https://a.thumbs.redditmedia.com/nYZXMaUA7KPZf...,140.0,"No need to panic, yer TP costs the same as thi...",14,/r/memes/comments/fhku3d/no_need_to_panic_yer_...


In [77]:
#df.to_json(r'Path to store the exported JSON file\File Name.json')
dank_data.to_json(r'./dank_tokenize.json')

In [78]:
# count rows in which value of 'text_from_image' == "Fail"
failed_link_memes = dank_data.apply(lambda x: True if x['text_from_image'] == 'Fail' else False , axis=1)
 
# Count number of True in series
numOfRows = len(failed_link_memes[failed_link_memes == True].index)
 
print('Number of Rows in dataframe with broken link : ', numOfRows)

Number of Rows in dataframe with broken link :  15913


In [79]:
# delete rows where value of 'text_from_image' == "Fail"

dank_data.drop(dank_data.index[dank_data['text_from_image'] == 'Fail'], inplace = True) 


In [80]:
dank_data

,author,awards,created_utc,text_from_image,words,processed_words,downs,id,is_nsfw,media,subreddit,subscribers,thumbnail.height,thumbnail.thumbnail,thumbnail.width,title,ups,url
1,L71U,[],1.584296e+09,NICE DICK\r\n,Ricardo😎NICE DICK\r\n,"[ricardo, nice, dick]",0,fj5ixk,False,https://i.redd.it/w6acm47gmvm41.jpg,r/dank_meme,743116,136.0,https://b.thumbs.redditmedia.com/PixKjoPWVw738...,140.0,Ricardo😎,35,/r/dank_meme/comments/fj5ixk/ricardo/
2,Awesome-ness5,[],1.584295e+09,Nvidia's calling on gaming PC owners to\r\nput...,Gamers AssembleNvidia's calling on gaming PC o...,"[gamer, assemblenvidia, call, game, owner, sys...",0,fj5bzj,False,https://i.redd.it/bqawbb5gkvm41.jpg,r/dank_meme,743116,140.0,https://b.thumbs.redditmedia.com/Fj8eeEt78J-I6...,140.0,Gamers Assemble,11,/r/dank_meme/comments/fj5bzj/gamers_assemble/
5,NaN,[],1.584294e+09,JANUARY\r\nFEBRUARY\r\nMARCH\r\nAPRIL\r\ncnu\r...,2020 in a nutshellJANUARY\r\nFEBRUARY\r\nMARCH...,"[nutshelljanuari, februari, march, april, plag...",0,fj53jo,False,https://i.imgur.com/7LmxYnC.png,r/dank_meme,743116,140.0,default,140.0,2020 in a nutshell,1,/r/dank_meme/comments/fj53jo/2020_in_a_nutshell/
7,notaspop,[],1.584294e+09,When you post a low-effort meme\r\nfor fun to ...,I mean.... Not Corona time?When you post a low...,"[mean, corona, time, post, effort, meme, peopl...",0,fj50df,False,https://i.redd.it/v5pea6wwgvm41.jpg,r/dank_meme,743116,140.0,default,140.0,I mean.... Not Corona time?,1,/r/dank_meme/comments/fj50df/i_mean_not_corona...
8,Gaunja420,[],1.584294e+09,H A p pyt1A\r\n,WheNH A p pyt1A\r\n,[whenh],0,fj501g,False,https://i.redd.it/809pja1tgvm41.gif,r/dank_meme,743116,140.0,default,140.0,WheN,1,/r/dank_meme/comments/fj501g/when/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53941,MightyTHR0G,[],1.584037e+09,CoronÄirus\r\nMy severely chapped hands\r\n,Wash them!CoronÄirus\r\nMy severely chapped ha...,"[wash, coronäirus, sever, chap, hand]",0,fhkua0,False,https://i.redd.it/ekegcqp5aam41.jpg,r/memes,9507605,140.0,https://b.thumbs.redditmedia.com/wsFjYB0tQx4WG...,140.0,Wash them!,22,/r/memes/comments/fhkua0/wash_them/
53943,Abk0306,[],1.584037e+09,Anti vax moms-\r\nYou can't defeat me.\r\nSoci...,Die mother fuckersAnti vax moms-\r\nYou can't ...,"[mother, fuckersanti, mom, defeat, societi, kn...",0,fhku7w,False,https://i.redd.it/2uuuxic5aam41.jpg,r/memes,9507605,140.0,https://b.thumbs.redditmedia.com/8dS66OUZe0asw...,140.0,Die mother fuckers,42,/r/memes/comments/fhku7w/die_mother_fuckers/
53944,ccanlas808,[],1.584037e+09,NaN,Fail,[fail],0,fhku7i,False,https://i.redd.it/3h9unvm5aam41.jpg,r/memes,9507605,140.0,default,140.0,“Just checking for COVID-19”,1,/r/memes/comments/fhku7i/just_checking_for_cov...
53946,M1CKYMC,[],1.584037e+09,ARSQUARE\r\nA 38680707 c\r\nmeme.org\r\n,"No need to panic, yer TP costs the same as thi...","[need, panic, cost, piecearsquar, meme]",0,fhku3d,False,https://i.redd.it/amea4y14aam41.jpg,r/memes,9507605,91.0,https://a.thumbs.redditmedia.com/nYZXMaUA7KPZf...,140.0,"No need to panic, yer TP costs the same as thi...",14,/r/memes/comments/fhku3d/no_need_to_panic_yer_...


In [81]:
# row in which value of 'text_from_image' column == "Fail"
failed_link_memes = dank_data.apply(lambda x: True if x['text_from_image'] == 'Fail' else False , axis=1)
 
# Count number of True in series
numOfRows = len(failed_link_memes[failed_link_memes == True].index)
 
print('Number of Rows in dataframe with broken link : ', numOfRows)

Number of Rows in dataframe with broken link :  0


In [82]:
#df.to_json(r'Path to store the exported JSON file\File Name.json')
dank_data.to_json(r'./dank_cleaned.json')